## Interactive Computing in Jupyter and Python with OmniSci    

![](https://avatars1.githubusercontent.com/u/7553829?s=200&v=4)    ![](https://avatars1.githubusercontent.com/u/34879953?s=200&v=4) 

### Bio

Tony Fast is a Developer Advocate at Quansight with a passion for literate programing and a contributor to Jupyter and HoloViz.

Kim Pevey is a Software Engineer at Quansight with an interest for analyzing big data with visualization tools in python. She is also a contributor to HoloViz.

### Abstract

This work discusses the open source technologies used to bring Omnisci to the greater scientific python ecosystem. These efforts allow Omnisci data to be analyzed in Jupyterlab, query data from omnisci databases, interoperate with `pandas` dataframes, and visualize information. We'll walk through several computational notebooks that cover the capabilities of Omnisci interacting with different data analysis and visualization tools in Python. Throughout these exercises we'll highlight the open source packages and technologies that amplify OmniSci's abilities with tools like `ibis`, `pandas`, and `holoviews`.

## Overview of OmniSci and the scientific python community


OmniSci is an analytics platform designed to handle very large datasets. It leverages the processing power of GPUs alongside traditional CPUs to achieve very high performance. OmniSci combines an open-source SQL engine (OmniSciDB), server-side rendering (OmniSci Render), and web-based data visualization (OmniSci Immerse) to provide a comprehensive platform for data analysis.


In [ ]:
    __import__("graphviz").Source("""graph {layout=circo 
    {{intake -- {ibis -- pandas}} -- {holoviews}}--hvplot geoviews--{holoviews hvplot}[style=dotted]}
    """)

<a id='available_tools'></a>
## Tools Available for Data Science

If you are an OmniSci open source edition user, you will not have access to Immerse, but you can still explore OmniSci with the Data Science Foundation tools.   

**Ibis / Pandas**  
Ibis is a productivity API for working in Python and analyzing data in remote SQL-based data stores such as OmniSciDB. Inspired by the pandas toolkit for data analysis, Ibis provides a Pythonic API that compiles to SQL. Combined with OmniSciDB scale and speed, Ibis offers a familiar but more powerful method for analyzing very large datasets "in-place."   
Ibis supports multiple SQL databases backends, and also supports pandas as a native backend. Combined with Altair, this integration allows you to explore multiple datasets across different data sources.

**Holoviews / hvPlot / GeoViews**  
HoloViews is an open-source Python library designed to make data analysis and visualization seamless and simple. With HoloViews, you can usually express what you want to do in very few lines of code, letting you focus on what you are trying to explore and convey, not on the process of plotting. hvPlot provides a high-level plotting API built on HoloViews that provides a general and consistent API for plotting data in common data formats such as pandas, xarray, dask, intake, NeworkX, and more. Through the Holoviews-Ibis-OmniSci workflow, its possible to interactively visualize OmniSci data without writing SQL code. GeoViews is a geographic extension of the Holoviews library which provides tools for projecting and visualizing geographic data. 

**Intake OmniSci Plugin**  
[Intake](https://intake.readthedocs.io) is a lightweight package for finding, investigating, loading and disseminating data. It is a cataloging system for listing data sources, their metadata and parameters, and referencing which of driver should load each. In short, [Intake](https://intake.readthedocs.io) allows the user to specify data sources via human-readable YAML catalogs, and then transparently load them and begin analyzing data.

<a id='ibis'></a>
## Introducing Ibis

> Write your analytics code once, run it everywhere.

In this example, we will be using Ibis to create and manage our connection to the database. Ibis will allow us to construct complex data anlytics using a Pandas-like API. It will convert our analytics methods to a SQL query, but will push the computational burden of the query to the server. In this way, users can query extremely large databases on remote servers without heavy local computation.  
  
For this example we'll use a local database running inside of docker.

Connect to the database using ibis:

In [ ]:
import ibis

omnisci_client = ibis.omniscidb.connect(
    user='demouser', 
    password='HyperInteractive',
    host='ships-demo-local.mapd.com', 
    port=443, 
    protocol='https',
    database='mapd'
)

### Exploring the database using Ibis

Let's use the client to take a look at the database.
We can quickly get a list of the tables available in the database.  

In [ ]:
omnisci_client.list_tables()

Now we will make a connection to the `ships_ais` table.

In [ ]:
ships = omnisci_client.table('ships_ais')
display(ships)
ships.head(2).execute()

You'll notice that when you inspect `ships` you see a schema object, not actual results.   
Ibis won't load the data and perform any computation, but instead will leave the data in the   
backend defined in the connection, and we will _ask_ the backend to perform the computations.  

This is a valuable tool when working with big data in which our client side cannot handle the  
volume of data until we have reduced it.  
  
Let's take a quick look at information Ibis has for this table without actually pulling the data locally:

#### get the table info

In [ ]:
display(ships.info())

#### get the table metadata

In [ ]:
display(ships.metadata())

Ibis is converting our expression into a SQL expression. Let's take a look at the actual SQL query.

In [ ]:
print(ships.head(1000).compile())

The table has 11.6 billion rows which we definitely can't handle locally so we can grab a subsample, then execute the query. This will bring us back the requested table as a Pandas DataFrame (we haven't asked it to perform any calculations yet). 

#### execute the query

In [ ]:
ships_df = ships.head(1000).execute()
print(f'Return Type: {type(ships_df)}')
ships_df

Now we can immediately continue our data analytics using a Pandas DataFrame (or GeoPandas GeoDataFrame) or we can modify our Ibis query to perform some calculations before pulling back data.  

<a id='advanced'></a>
## Advanced Ibis Queries

We can use Ibis to construct complex SQL queries and push the computation required for these queries to the server. This puts the computational burden on the server rather than the local machine, and allows for users to transform and reduce the dataset before
bringing the results back locally.  
  
For example, we can aggregate the number of vessels by grouping on the Vessel Type. The final `execute` statement pushes this computation to be calculated on the server and returns a Pandas DataFrame

In [ ]:
expr = ships.groupby('VesselType').aggregate(count=ships.VesselType.count()).sort_by(('count', False)).limit(20)
print(expr.compile())
expr

In [ ]:
vessel_counts = expr.execute()
vessel_counts

We can easily visualize this pandas dataframe in holoviews

In [ ]:
import holoviews as hv
hv.extension('bokeh')
hv.Bars(vessel_counts).opts(xrotation=90, width=900, height=400)

But we also can simple pass the `ibis` expression to Holoviews and it can process it. 

In [ ]:

hv.Bars(expr).sort('count', reverse=True).compute().opts(xrotation=90, width=900, height=400)

Next, we can filter out the data from a single vessel. This is fairly time consuming, but the computational burden is on the server. 

In [ ]:
name = 'SPIRIT OF ENDEAVOUR'
df = ships.filter(ships['unused_Name'] == name).execute()
print(len(df))

Our reduced dataset has 130k data points. Let's plot it and see where the *Spirit of Endeavour* has been.

We can use GeoViews, which is a geographic extension of the Holoviews library. This will allow us to bring in some background imagery and it will also (silently) help us out with projection. 

### plotting data onto maps

`geoviews` contains tools for different web map tile services.

In [ ]:
hv.element.tiles.StamenWatercolor() + hv.element.tiles.Wikipedia() + hv.element.tiles.ESRI()

In [ ]:
import geoviews as gv
import geoviews.tile_sources as gvts
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
import holoviews as hv

In [ ]:
gvts.StamenTerrain * dynspread(datashade(gv.Points(df, kdims=['Longitude','Latitude']), cmap=["red"])).opts(width=900, height=600)

Before moving on, we'll close our connection to the database.

In [ ]:
omnisci_client.close()